In [46]:
import pandas as pd
import time
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [47]:
df_dev  = pd.read_csv("../dados/ready/dev.csv")
df_comp = pd.read_csv("../dados/ready/comp.csv")

In [48]:
def get_x(df: pd.DataFrame) -> None:
    condition = (df.columns != 'status') & (df.columns != 'account_id') & (df.columns != 'loan_id') & (
        df.columns != 'client_id') & (df.columns != 'district_id') & (df.columns != 'disp_id') & (df.columns != 'district_id_y') & (
        df.columns != 'district_id_x') & (df.columns != 'id')
    return df.loc[:, condition]

def get_y(df: pd.DataFrame) -> None:
    return df.status

In [144]:
start = time.time()

train, test = train_test_split(df_dev, test_size = 0.2)

x = get_x(train)
y = get_y(train)

log_reg = DecisionTreeClassifier(max_depth=12)
log_reg.fit(x, y)

end = time.time()

pred_competition = log_reg.predict_proba(get_x(df_comp))

prediction = pd.DataFrame({'Id': df_comp['loan_id'], 'Predicted': pred_competition[::, 1]})

predicted = log_reg.predict_proba(get_x(test))[::, 1]
expected  = get_y(test)
score = roc_auc_score(expected, predicted)

print(f"Score is {score}")

print(f"Time Elapsed: {end-start}")

Score is 0.695906432748538
Time Elapsed: 0.019004344940185547


In [64]:
prediction.to_csv("../predictions/decision-tree.csv", index = False)